In [1]:
import pandas as pd
import numpy as np
from sql_functions import *
import plotly.express as px
import json
from normal_round import normal_round as normal_round

In [2]:
pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
df_laender_abs = get_dataframe('SELECT * FROM public.laender_jugend_tat_absolut_2022_until_2018;')
df_laender_rel = get_dataframe('SELECT * FROM public.laender_jugend_tat_relativ_2022_until_2018;')

In [4]:
df_laender_abs.head(5)

,schluessel,straftat,bundesland,sexus,tatverdaechtige_insgesamt,kinder_10_bis_unter_12,kinder_12_bis_unter_14,kinder_10_bis_unter_14,kinder_unter_14,jugendliche_14_bis_unter_16,jugendliche_16_bis_unter_18,jugendliche_14<18,heranwachsende_18_bis_unter_21,jugendl_u_heranwachsende_14_bis_unter_21,erwachsene_ab_21,year
0,------,Straftaten insgesamt,Baden-Württemberg,M,184801,1305,2975,4280,5246,6585,10078,16663,20122,36785,142770,2018
1,------,Straftaten insgesamt,Bayern,M,226208,1390,3101,4491,6276,6786,10604,17390,24065,41455,178477,2018
2,------,Straftaten insgesamt,Berlin,M,101377,831,1561,2392,3112,2881,3977,6858,7941,14799,83466,2018
3,------,Straftaten insgesamt,Brandenburg,M,49670,547,962,1509,2050,1869,2469,4338,3910,8248,39372,2018
4,------,Straftaten insgesamt,Bremen,M,18519,95,224,319,389,642,1125,1767,1934,3701,14429,2018


In [5]:
#df_laender_rel.tail()

### Calculate most occuring crime per state - rank and percentage of total

In [6]:
df_1 = df_laender_abs.copy()
#df_1 = df_1[df_1['sexus'] == 'X']
df_1 = df_1[['schluessel', 'straftat', 'bundesland', 'sexus', 'year', 'jugendliche_14_bis_unter_16']]
df_1 = df_1.rename(columns={'jugendliche_14_bis_unter_16': 'offenders'})

df_2 = df_1[df_1['schluessel'] == '------']
df_2 = df_2[['bundesland', 'sexus', 'year', 'offenders']]
df_laender_straftaten_insg_absolut = df_2.copy() # needed later

df_3 = df_1[df_1['schluessel'] != '------']

df_3['rank'] = (
    df_3.groupby(['bundesland', 'sexus', 'year'])['offenders']
        .rank(ascending=False, method='first')
        .astype(int)
)

df_4 = df_3.merge(df_2, on=['bundesland', 'sexus', 'year'], how='left')

df_4['certain_crime_percent_of_total_crime'] = df_4['offenders_x'] / df_4['offenders_y'] * 100
df_4 = df_4.drop(columns='offenders_y')
df_4 = df_4.reset_index()
df_4 = df_4.drop(columns='index')

values = df_4['certain_crime_percent_of_total_crime'].tolist()
new_values = []
for v in values:
    new_v = normal_round(num=v, ndigits=2)
    new_values.append(new_v)
    
df_4['certain_crime_percent_of_total_crime'] = new_values

df_4 = df_4[['schluessel', 'bundesland', 'sexus', 'year', 'straftat', 'rank', 'certain_crime_percent_of_total_crime']]

crime_rank_and_percent_of_total_per_state = df_4.copy()
crime_rank_and_percent_of_total_per_state.head()

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14901/1686970950.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['rank'] = (


,schluessel,bundesland,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
0,100000,Baden-Württemberg,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,4.92
1,100000,Bayern,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,5.85
2,100000,Berlin,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6,3.92
3,100000,Brandenburg,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,5.62
4,100000,Bremen,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,4.05


### Create overview - number of offenders per state - absolut and relativ

In [7]:
df_10 = df_laender_rel.copy()
df_10 = df_10.rename(columns={'jugendliche_14_bis_unter_16_rel': 'offenders_rel'})
df_10 = df_10[(df_10['schluessel'] == '------')]

jugendkrim_insg_rel = df_10['offenders_rel'].tolist()
jugendkrim_insg_rel_rounded = []
for v in jugendkrim_insg_rel:
    v_new = int(normal_round(v, 0))
    jugendkrim_insg_rel_rounded.append(v_new)
df_10['offenders_rel'] = jugendkrim_insg_rel_rounded

df_10 = df_10[['bundesland', 'sexus', 'year', 'offenders_rel']]
df_10 = df_10.reset_index()
df_10 = df_10.drop(columns='index')

df_11 = df_laender_straftaten_insg_absolut.merge(df_10, on=['bundesland', 'sexus', 'year'], how='left')
df_laender_straftaten_insg_abs_u_rel = df_11.copy()
df_laender_straftaten_insg_abs_u_rel.head()

,bundesland,sexus,year,offenders,offenders_rel
0,Baden-Württemberg,M,2018,6585,6007
1,Bayern,M,2018,6786,5534
2,Berlin,M,2018,2881,10002
3,Brandenburg,M,2018,1869,8664
4,Bremen,M,2018,642,10819


### Merge crimes that <br>rank on 1, 2, and 3 <br>and <br>overall number of crimes - absolut and relativ <br>to every state for every year (->one DataFrame)

In [8]:
df_20 = crime_rank_and_percent_of_total_per_state.copy()
df_rank_1 = df_20[df_20['rank'] == 1]
df_rank_2 = df_20[df_20['rank'] == 2]
df_rank_3 = df_20[df_20['rank'] == 3]
df_rank_4 = df_20[df_20['rank'] == 4]
df_rank_5 = df_20[df_20['rank'] == 5]
df_rank_1.head()

,schluessel,bundesland,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
98,220000,Berlin,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,26.31
99,220000,Brandenburg,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,23.92
103,220000,Mecklenburg-Vorpommern,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,24.22
104,220000,Niedersachsen,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,23.68
105,220000,Nordrhein-Westfalen,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,28.76


In [9]:
df_30 = df_laender_straftaten_insg_abs_u_rel.copy()

df_31 = df_30.merge(df_rank_1, on=['bundesland', 'sexus', 'year'], how='left')
df_31 = df_31.rename(columns={'straftat': 'crimes_on_rank_1',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_1_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_1'})
df_31 = df_31.drop(columns='rank')

df_32 = df_31.merge(df_rank_2, on=['bundesland', 'sexus', 'year'], how='left')
df_32 = df_32.rename(columns={'straftat': 'crimes_on_rank_2',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_2_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_2'})
df_32 = df_32.drop(columns='rank')

df_33 = df_32.merge(df_rank_3, on=['bundesland', 'sexus', 'year'], how='left')
df_33 = df_33.rename(columns={'straftat': 'crimes_on_rank_3',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_3_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_3'})
df_33 = df_33.drop(columns='rank')

df_34 = df_33.merge(df_rank_4, on=['bundesland', 'sexus', 'year'], how='left')
df_34 = df_34.rename(columns={'straftat': 'crimes_on_rank_4',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_4_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_4'})
df_34 = df_34.drop(columns='rank')

df_35 = df_34.merge(df_rank_5, on=['bundesland', 'sexus', 'year'], how='left')
df_35 = df_35.rename(columns={'straftat': 'crimes_on_rank_5',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_5_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_5'})
df_35 = df_35.drop(columns='rank')

df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5 = df_35.copy() 
df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.head()

,bundesland,sexus,year,offenders,offenders_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,Baden-Württemberg,M,2018,6585,6007,*26*00,Ladendiebstahl insgesamt,27.11,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.52,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),17.97,674000,Sachbeschädigung §§ 303-305a StGB,13.77,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.92
1,Bayern,M,2018,6786,5534,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),21.01,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.79,*26*00,Ladendiebstahl insgesamt,20.31,674000,Sachbeschädigung §§ 303-305a StGB,14.10,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.85
2,Berlin,M,2018,2881,10002,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",26.31,*26*00,Ladendiebstahl insgesamt,26.03,674000,Sachbeschädigung §§ 303-305a StGB,11.80,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",8.16,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),6.94
3,Brandenburg,M,2018,1869,8664,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",23.92,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),20.06,*26*00,Ladendiebstahl insgesamt,18.89,674000,Sachbeschädigung §§ 303-305a StGB,18.83,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.62
4,Bremen,M,2018,642,10819,*26*00,Ladendiebstahl insgesamt,25.08,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",19.00,674000,Sachbeschädigung §§ 303-305a StGB,8.26,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),6.70,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.05


### Calculate Growth Rate

In [10]:
df_51 = df_laender_abs.copy()
df_51 = df_51[["schluessel", "straftat", "bundesland", "year", 'sexus', "jugendliche_14_bis_unter_16"]]
df_51 = df_51[df_51["schluessel"] == "------"]
df_51 = df_51[["bundesland", "year", 'sexus', "jugendliche_14_bis_unter_16"]]

df_54 = pd.DataFrame(columns = ["bundesland", "year", 'sexus', "jugendliche_14_bis_unter_16", "growth_rate_abs"])

gender = ['X', 'W', 'M']
bundeslaender = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
for g in gender:
    df_52 = df_51[df_51["sexus"] == g]
    for b in bundeslaender:
        df_53 = df_52[df_52["bundesland"] == b]
        df_53["growth_rate_abs"] = df_53['jugendliche_14_bis_unter_16'].pct_change() * 100
        df_54 = pd.concat([df_54, df_53])

df_54 = df_54.drop(columns='jugendliche_14_bis_unter_16')

df_laender_growth_rate_abs = df_54.copy()
df_laender_growth_rate_abs.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14901/78520704.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_53["growth_rate_abs"] = df_53['jugendliche_14_bis_unter_16'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14901/78520704.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_53["growth_rate_abs"] = df_53['jugendliche_14_bis_unter_16'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14901/78520704.py:17: SettingWith

,bundesland,year,sexus,growth_rate_abs
32,Baden-Württemberg,2018,X,NaN
560,Baden-Württemberg,2019,X,4.838203
1088,Baden-Württemberg,2020,X,-12.136650
1616,Baden-Württemberg,2021,X,-2.444293
2144,Baden-Württemberg,2022,X,29.262324


In [11]:
df_61 = df_laender_rel.copy()
df_61 = df_61[["schluessel", "straftat", "bundesland", "year", 'sexus', "jugendliche_14_bis_unter_16_rel"]]
df_61 = df_61[df_61["schluessel"] == "------"]
df_61 = df_61[["bundesland", "year", 'sexus', "jugendliche_14_bis_unter_16_rel"]]

df_64 = pd.DataFrame(columns = ["bundesland", "year", 'sexus', "jugendliche_14_bis_unter_16_rel", "growth_rate_rel"])

gender = ['X', 'W', 'M']
bundeslaender = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
for g in gender:
    df_62 = df_61[df_61["sexus"] == g]
    for b in bundeslaender:
        df_63 = df_62[df_62["bundesland"] == b]
        df_63["growth_rate_rel"] = df_63['jugendliche_14_bis_unter_16_rel'].pct_change() * 100
        df_64 = pd.concat([df_64, df_63])
        
df_64 = df_64.drop(columns='jugendliche_14_bis_unter_16_rel')

df_laender_growth_rate_rel = df_64.copy()
df_laender_growth_rate_rel.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14901/2290470051.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_63["growth_rate_rel"] = df_63['jugendliche_14_bis_unter_16_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14901/2290470051.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_63["growth_rate_rel"] = df_63['jugendliche_14_bis_unter_16_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14901/2290470051.py:1

,bundesland,year,sexus,growth_rate_rel
32,Baden-Württemberg,2018,X,NaN
560,Baden-Württemberg,2019,X,5.479344
1088,Baden-Württemberg,2020,X,-11.709984
1616,Baden-Württemberg,2021,X,-0.899808
2144,Baden-Württemberg,2022,X,29.055751


In [12]:
df_growth_rate_14_to_under_16 = df_laender_growth_rate_abs.merge(df_laender_growth_rate_rel, on=['bundesland', 'year', 'sexus'], how='left')
df_growth_rate_14_to_under_16.head()

,bundesland,year,sexus,growth_rate_abs,growth_rate_rel
0,Baden-Württemberg,2018,X,NaN,NaN
1,Baden-Württemberg,2019,X,4.838203,5.479344
2,Baden-Württemberg,2020,X,-12.136650,-11.709984
3,Baden-Württemberg,2021,X,-2.444293,-0.899808
4,Baden-Württemberg,2022,X,29.262324,29.055751


In [13]:
values_growth_rate_abs = df_growth_rate_14_to_under_16["growth_rate_abs"].tolist()
new_values_abs = []
for v in values_growth_rate_abs:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_abs.append(new_v)
df_growth_rate_14_to_under_16["growth_rate_abs"] = new_values_abs

values_growth_rate_rel = df_growth_rate_14_to_under_16["growth_rate_rel"].tolist()
new_values_rel = []
for v in values_growth_rate_rel:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_rel.append(new_v)
df_growth_rate_14_to_under_16["growth_rate_rel"] = new_values_rel

In [14]:
df_growth_rate_14_to_under_16.head()

,bundesland,year,sexus,growth_rate_abs,growth_rate_rel
0,Baden-Württemberg,2018,X,NaN,NaN
1,Baden-Württemberg,2019,X,4.84,5.48
2,Baden-Württemberg,2020,X,-12.13,-11.70
3,Baden-Württemberg,2021,X,-2.43,-0.89
4,Baden-Württemberg,2022,X,29.26,29.06


In [15]:
df_overview_fed_states_14_to_under_16 = df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.merge(df_growth_rate_14_to_under_16, on=['bundesland', 'sexus', 'year'], how='left')
#df_overview_fed_states = df_overview_fed_states.drop(columns='sexus')

df_overview_fed_states_14_to_under_16['age_group'] = 'jugendliche_14_bis_unter_16'

# bring in correct order
df_overview_fed_states_14_to_under_16 = df_overview_fed_states_14_to_under_16[['bundesland', 'year', 'sexus', 'age_group', 'offenders', 'offenders_rel',
       'growth_rate_abs', 'growth_rate_rel',
       'schluessel_crimes_on_rank_1', 'crimes_on_rank_1',
       'percentage_of_rank_1_on_crime_total', 'schluessel_crimes_on_rank_2',
       'crimes_on_rank_2', 'percentage_of_rank_2_on_crime_total',
       'schluessel_crimes_on_rank_3', 'crimes_on_rank_3',
       'percentage_of_rank_3_on_crime_total', 'schluessel_crimes_on_rank_4',
       'crimes_on_rank_4', 'percentage_of_rank_4_on_crime_total',
       'schluessel_crimes_on_rank_5', 'crimes_on_rank_5',
       'percentage_of_rank_5_on_crime_total']]

df_overview_fed_states_14_to_under_16.tail(5)

,bundesland,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
235,Saarland,2022,X,jugendliche_14_bis_unter_16,1196,7420,37.47,38.14,*26*00,Ladendiebstahl insgesamt,30.27,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",25.75,674000,Sachbeschädigung §§ 303-305a StGB,9.36,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8.19,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),8.03
236,Sachsen,2022,X,jugendliche_14_bis_unter_16,4190,6059,33.48,29.72,*26*00,Ladendiebstahl insgesamt,26.16,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",16.06,674000,Sachbeschädigung §§ 303-305a StGB,14.46,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,10.07,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),9.81
237,Sachsen-Anhalt,2022,X,jugendliche_14_bis_unter_16,2720,7654,25.58,22.71,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",23.79,*26*00,Ladendiebstahl insgesamt,21.18,674000,Sachbeschädigung §§ 303-305a StGB,16.25,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),13.20,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8.13
238,Schleswig-Holstein,2022,X,jugendliche_14_bis_unter_16,3181,5915,24.94,23.21,*26*00,Ladendiebstahl insgesamt,30.37,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.22,674000,Sachbeschädigung §§ 303-305a StGB,10.15,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),9.87,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8.14
239,Thüringen,2022,X,jugendliche_14_bis_unter_16,2101,5981,26.11,22.63,*26*00,Ladendiebstahl insgesamt,22.99,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.70,674000,Sachbeschädigung §§ 303-305a StGB,14.61,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),11.09,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,11.04


In [16]:
df_overview_fed_states_14_to_under_16 = df_overview_fed_states_14_to_under_16.fillna('n.a.')

In [17]:
df_overview_fed_states_14_to_under_16.head()

,bundesland,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,Baden-Württemberg,2018,M,jugendliche_14_bis_unter_16,6585,6007,n.a.,n.a.,*26*00,Ladendiebstahl insgesamt,27.11,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.52,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),17.97,674000,Sachbeschädigung §§ 303-305a StGB,13.77,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.92
1,Bayern,2018,M,jugendliche_14_bis_unter_16,6786,5534,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),21.01,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.79,*26*00,Ladendiebstahl insgesamt,20.31,674000,Sachbeschädigung §§ 303-305a StGB,14.10,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.85
2,Berlin,2018,M,jugendliche_14_bis_unter_16,2881,10002,n.a.,n.a.,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",26.31,*26*00,Ladendiebstahl insgesamt,26.03,674000,Sachbeschädigung §§ 303-305a StGB,11.80,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",8.16,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),6.94
3,Brandenburg,2018,M,jugendliche_14_bis_unter_16,1869,8664,n.a.,n.a.,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",23.92,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),20.06,*26*00,Ladendiebstahl insgesamt,18.89,674000,Sachbeschädigung §§ 303-305a StGB,18.83,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.62
4,Bremen,2018,M,jugendliche_14_bis_unter_16,642,10819,n.a.,n.a.,*26*00,Ladendiebstahl insgesamt,25.08,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",19.00,674000,Sachbeschädigung §§ 303-305a StGB,8.26,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),6.70,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.05


# Now Create same DataFrame for other age groups and then concat them to df_overview_fed_states

In [18]:
%store df_overview_fed_states_14_to_under_16

Stored 'df_overview_fed_states_14_to_under_16' (DataFrame)
